In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import torch
import time
import random
import spacy
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import warnings

from torchtext.legacy import data
from torchtext.legacy import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
warnings.filterwarnings("ignore")

In [2]:
seed = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'

TEXT = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(sequential=False, use_vocab = False)

df = pd.read_csv("drive/MyDrive/ire_major/clean.csv")
df.rename(columns={"from":"author"}, inplace = True)

df["text"].replace('', np.nan, inplace = True)
df.dropna(inplace = True)

df["length"] = df["text"].apply(lambda x: len(x.split()))
df = df.drop(df[df.length > 500].index)
df = df.drop(df[df.length < 10].index)

items = df.author.value_counts().to_dict().items()
df = df[df.author.isin([key for key, val in items if val > 2000])]

texts = df.text.tolist()
labels = df.author.tolist()
label2id = {i: idx for (idx, i) in enumerate(sorted(set(labels)))}
id2label = {label2id[i]: i for i in label2id}
labels = [label2id[i] for i in labels]

In [38]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.15, random_state = 42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.15, random_state = 42)

train_df_dict = {'text':train_texts, 'label':train_labels}
test_df_dict = {'text':test_texts, 'label':test_labels}
val_df_dict = {'text':val_texts, 'label':val_labels}

train_df = pd.DataFrame.from_dict(train_df_dict)
test_df = pd.DataFrame.from_dict(test_df_dict)
val_df = pd.DataFrame.from_dict(val_df_dict)

train_df.to_csv('enron_train.csv', index =  False, header = False)
test_df.to_csv('enron_test.csv', index = False, header = False)
val_df.to_csv('enron_val.csv', index = False, header = False)

train_data, val_data, test_data = data.TabularDataset.splits(
    path='./', train='enron_train.csv', validation = 'enron_val.csv', test = 'enron_test.csv', format='csv',
    fields=[('text', TEXT), ('label', LABEL)])

TEXT.build_vocab(train_data, vectors = "glove.6B.50d", unk_init = torch.Tensor.normal_)

train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data), batch_sizes=(16, 16, 16),
    sort_key=lambda x: len(x.text), device=device)

In [39]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):        
        text = text.permute(1, 0)        
        embedded = self.embedding(text)        
        embedded = embedded.unsqueeze(1)        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

In [40]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = 15
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [41]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

optimizer = optim.AdamW(model.parameters())
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)

In [42]:
def compute_metrics(preds, labels):
    preds = preds.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return acc, f1, precision, recall

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0

    model.train()
    
    for batch in iterator:        
        optimizer.zero_grad()

        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        acc, f1, precision, recall = compute_metrics(predictions.cpu(), batch.label.cpu())
        loss.backward()
        
        optimizer.step()        

        epoch_loss += loss
        epoch_acc += acc
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator)

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0

    model.eval()  
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            acc, f1, precision, recall = compute_metrics(predictions.cpu(), batch.label.cpu())            
            epoch_loss += loss
            epoch_acc += acc
            epoch_f1 += f1
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [43]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc, train_f1 = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, val_iter, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    torch.save({'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, f'models/enron_model_{epoch}')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train F1: {train_f1}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. F1: {valid_f1}' )


Epoch: 01 | Epoch Time: 0m 32s
	Train Loss: 0.839 | Train Acc: 74.50% | Train F1: 0.7378487714557711
	 Val. Loss: 0.468 |  Val. Acc: 86.42% | Val. F1: 0.8553815430484548
Epoch: 02 | Epoch Time: 0m 32s
	Train Loss: 0.437 | Train Acc: 87.20% | Train F1: 0.8691401968187605
	 Val. Loss: 0.306 |  Val. Acc: 91.07% | Val. F1: 0.9041025592463089
Epoch: 03 | Epoch Time: 0m 32s
	Train Loss: 0.314 | Train Acc: 91.07% | Train F1: 0.908718141399236
	 Val. Loss: 0.241 |  Val. Acc: 93.11% | Val. F1: 0.9275917221667218
Epoch: 04 | Epoch Time: 0m 32s
	Train Loss: 0.236 | Train Acc: 93.46% | Train F1: 0.9332525872488318
	 Val. Loss: 0.190 |  Val. Acc: 94.80% | Val. F1: 0.9438053217615716
Epoch: 05 | Epoch Time: 0m 32s
	Train Loss: 0.185 | Train Acc: 95.07% | Train F1: 0.9494021206880092
	 Val. Loss: 0.153 |  Val. Acc: 95.96% | Val. F1: 0.9574320984570985


In [44]:
test_loss, test_acc, test_f1 = evaluate(model, test_iter, criterion)

In [45]:
test_f1

0.9519429750615976